\*\* *(under construction. If you're using Binder, please note that this tutorial is only fully functional in Firefox at the moment.)* \*\*

Adapted from [a notebook](https://nbviewer.jupyter.org/github/marksubbarao/pyWWT_AAS225/blob/master/CosmicFlows.ipynb) by Mark SubbaRao that plots galactic data from [Tully et al. 2013](https://iopscience.iop.org/article/10.1088/0004-6256/146/4/86/meta).

In [ ]:
import time
import numpy as np

from astropy.coordinates import SkyCoord, Distance
from astropy import units as u
from astropy.table import Table, Column
from astropy.time import Time, TimeDelta

from astroquery.vizier import Vizier

from matplotlib import colors
import matplotlib.pyplot as plt

from pywwt.jupyter import WWTJupyterWidget

In [ ]:
wwt = WWTJupyterWidget()
wwt

*WWT works under HTTP, while Binder uses HTTPS. As such, in order to access all features of pywwt, Binder users will need to disable HTTPS for this session. Firefox users can do this with the following steps:*
- *Click the lock icon to the left of the address bar,*
- *Select the arrow next to “Connection,”*
- *Then, press the “Disable protection for now” button. (Protection is disabled for this page and only during this session, not for your whole browser. We are working on upgrading WWT so this step is no longer necessary.)*

## Visualizing the *Cosmicflows-2* galaxy catalog

(In this more simple example, we plot several galaxies in 3-D and color them by their velocities.)

**First, we use a class from the `astroquery` package to query the Vizier catalog and return an `astropy` table that contains redshift-independent distances, peculiar velocities, and coordinates for thousnads of galaxies.** 

We begin by slimming down the table to the columns necessary for this example.

In [ ]:
v = Vizier()
v.ROW_LIMIT = -1

In [ ]:
cats = v.get_catalogs('J/AJ/146/86')
cosmic_flows = cats[1]
cosmic_flows.keep_columns(["GLON", "GLAT", "__Dist_",
                           "__PV_", "__Bmag_","N1"])
cosmic_flows.rename_column('__Dist_', 'Dist')
cosmic_flows.rename_column('__PV_', 'PV')
cosmic_flows.rename_column('__Bmag_', 'Bmag')

*(Open a new cell above this one and see what the table looks like by entering `cosmic_flows` and running it.)*

~~**Next, since the latitude and longitude coordinates are in the galactic frame, we use `SkyCoord` from `astropy` to convert them to right ascension and declination, then add those columns to our table.**~~

In [ ]:
#coords = SkyCoord(cosmic_flows['GLON'], cosmic_flows['GLAT'],
#                  unit=(u.deg, u.deg),
#                  distance=Distance(cosmic_flows['Dist'],u.Mpc),
#                 )#frame='galactic')

In [ ]:
#cosmic_flows.remove_columns(['RA', 'Dec'])

#cosmic_flows.add_column(Column(coords.icrs.ra, name='ra'))
#cosmic_flows.add_column(Column(coords.icrs.dec, name='dec'))

**`pywwt` can read colormap columns from tables when plotting points, so we'll continue by assigning colors to each galaxy.**

`plt.cm` contains a number of pre-made colormap objects for us to choose from. To illustrate the peculiar velocity (`'PV'`) of these galaxies, I chose `magma`, whose color transition evokes thoughts of red & blueshift and should also be accessible to people with common forms of colorblindness.

In [ ]:
# try twilight shifted, but also remember r/b colorblind accessibility
cmap = plt.cm.magma(cosmic_flows['PV'])

In [ ]:
#norm = plt.Normalize(min(cosmic_flows['PV']), max(cosmic_flows['PV']))
#colors2 = plt.cm.magma(norm(cosmic_flows['PV']))

After getting an array of RGB colors, we convert them to hex codes and add a colormap column to our table.

In [ ]:
cosmic_flows['cmap'] = '#000000'
for i, _ in enumerate(cosmic_flows['PV']):
    cosmic_flows['cmap'][i] = colors.rgb2hex(cmap[i])

Re-run the cell where you examined `cosmic_flows` to see the new columns we've added to the table.

In [ ]:
cosmic_flows

**We're now ready to work in `pywwt`. Move the cell containing the `wwt` viewer down here. Then, switch to the solar system view so we can plot the galaxies in three-dimensional space.**

In [ ]:
wwt.set_view('solar system')

**Next, since the longitude and latitude columns of the table are in the galactic frame, we turn on `wwt`'s galactic mode.**

To improve loading times, we also turn off the COSMOS galactic survey that normally loads in the background.

In [ ]:
wwt.galactic_mode = True
wwt.solar_system.cosmos = False

**Finally, we're ready to plot a data layer based on the table we've modified.**

We link the appropriate columns for longitude, latitude, altitude, and colormap. Since our "altitude" in this scenario is distance of the galaxy from our perspective, we set `alt_type` to `'distance'`.

In [ ]:
gals = wwt.layers.add_data_layer(table=cosmic_flows, frame='Sky',
                                 #lon_att='ra', lat_att='dec',
                                 lon_att='GLON', lat_att='GLAT',
                                 alt_att='Dist', alt_type='distance',
                                 cmap_att='cmap', size_scale=40)

**Zoom out until you've reached galactic scale and see the thousands of galaxies points we've plotted.**

It's also possible to edit a layer's attributes after its creation. If you would like to see all points even when they're on the far side, run the following cell:

In [ ]:
gals.far_side_visible = True